In [1]:
import os

os.environ["OPENAI_API_KEY"] = "your_openai_key_here"
os.environ["GEMINI_API_KEY"] = "your_gemini_key_here"
os.environ["GOOGLE_API_KEY"] = "your_google_key_here"
os.environ["DEEPSEEK_API_KEY"] = "your_deepseek_key_here"
os.environ['TAVILY_API_KEY'] = "your_tavily_key_here"

In [2]:
import asyncio
import os
import re
import nest_asyncio
from urllib.parse import urlparse
from gpt_researcher import GPTResearcher

nest_asyncio.apply()

# === Optional: Toggle Ollama LLM use ===
USE_OLLAMA = False # Set to False to use default LLMs (e.g., OpenAI)

if USE_OLLAMA:
    os.environ["OLLAMA_BASE_URL"] = "http://127.0.0.1:11434"
    
    # for fast operations like summaries
    os.environ["FAST_LLM"] = "ollama:mistral:7b-instruct" 
    # # for smart operations like generating reports and reasoning
    os.environ["SMART_LLM"] = "deepseek:deepseek-chat"
    # # for strategic operations like generating research plans
    os.environ["STRATEGIC_LLM"] = "ollama:mistral-nemo:12b" 

    os.environ["EMBEDDING"] = "ollama:nomic-embed-text"
else:
    for var in ["OLLAMA_BASE_URL", "FAST_LLM", "SMART_LLM", "STRATEGIC_LLM", "EMBEDDING"]:
        os.environ.pop(var, None)


In [3]:

### import asyncio
import os
import re
import nest_asyncio
from urllib.parse import urlparse
from gpt_researcher import GPTResearcher
import logging

nest_asyncio.apply()

# === Load prompts from SESSION14.txt ===
def load_prompts(filepath):
    with open(filepath, "r") as f:
        text = f.read()

    pattern = r"Prompt\s+(\d+):\s*(.*?)\s*(?=Prompt\s+\d+:|$)"
    matches = re.findall(pattern, text, re.DOTALL)
    return [(num.zfill(3), prompt.strip()) for num, prompt in matches]

# === Extract domain from URL ===
def extract_domain(url):
    return urlparse(url).netloc

# === Run GPTResearcher for one prompt ===
async def get_report(prompt_text: str, report_type: str = "deep"):
    researcher = GPTResearcher(prompt_text, report_type)
    await researcher.conduct_research()
    report = await researcher.write_report()

    # Extract visited domains
    sources = researcher.get_research_sources()
    domains = list(dict.fromkeys(
        extract_domain(src["url"])
        for src in sources
        if isinstance(src, dict) and "url" in src
    ))
    return report, domains

system_message = """
You are a professional research assistant tasked with preparing a structured, data-driven report on the topic the user provides. Your goal is to deliver clear, factual, and well-cited analysis.

Do:
- Focus on data-rich insights: include specific figures, trends, statistics, and measurable outcomes (e.g., market size, historical trends, cost comparisons, adoption rates).
- When appropriate, summarize data in a way that could be turned into visualizations or tables (e.g., “this would work well as a bar chart comparing regional adoption rates”).
- Prioritize reliable, up-to-date sources: peer-reviewed research, government and international organizations, industry white papers, or credible journalism.
- Include inline citations and return all source metadata used in the report.

Be analytical, avoid vague statements, and ensure each section supports reasoned, evidence-backed conclusions that could inform decision-making or future research.\n\n
"""

# === Run all prompts from SESSION14.txt ===
async def run_all_prompts(topic_file="prompts/SESSION14.txt", output_dir="gpt-research/dd16_gpt4"):
    prompts = load_prompts(topic_file)
    os.makedirs(output_dir, exist_ok=True)

    for idx, prompt in prompts:
        prompt = system_message + prompt 
        if int(idx) < 40:
            continue  # Skip prompts before index 030
            
        report_path = os.path.join(output_dir, f"report_{idx}.txt")
        domain_path = os.path.join(output_dir, f"domains_{idx}.txt")
        log_path = os.path.join(output_dir, f"log_{idx}.txt")   

        # Skip if report already exists
        if os.path.exists(report_path) and os.path.exists(domain_path):
            print(f"✅ Skipping Prompt {idx} (already exists)")
            continue
        
        # get GPTResearcher internal loggers
        gpt_logger = logging.getLogger("gpt_researcher")
        scraper_logger = logging.getLogger("gpt_researcher.scraper")
        
        # remove any old handlers to avoid duplicate logs
        for lg in [gpt_logger, scraper_logger]:
            for h in list(lg.handlers):
                lg.removeHandler(h)
            lg.setLevel(logging.INFO)
            lg.propagate = False
        
        # attach a file handler per prompt
        fh = logging.FileHandler(log_path)
        fh.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
        for lg in [gpt_logger, scraper_logger]:
            lg.addHandler(fh)

        print(f"Running Prompt {idx}...")
        try:
            report, domains = await get_report(prompt)
            if len(domains) > 0:
                with open(domain_path, "w") as f:
                    for d in domains:
                        f.write(d + "\n")

            with open(report_path, "w") as f:
                f.write(report)

        except Exception as e:
            print(f"❌ Error on Prompt {idx}: {e}")

# === Run the script ===
if __name__ == "__main__":
    asyncio.run(run_all_prompts("prompts/DD16.txt"))


✅ Skipping Prompt 040 (already exists)
✅ Skipping Prompt 041 (already exists)
Running Prompt 042...
Error: 400 Client Error: Bad Request for url: https://api.tavily.com/search. Failed fetching sources. Resulting in empty response.


INFO:     [15:00:15] 🔍 Starting the research task for '“Arctic sea ice extent, thickness, and seasonality trends 2000–2023 AND offshore mineral exploration activities (survey campaigns, initial drilling trials, site selection) for rare earth elements, nickel, copper in Barents, Chukchi, Beaufort”'...
INFO:     [15:00:15] 🌎 Environmental & Energy Research Agent
INFO:     [15:00:15] 🌐 Browsing the web to learn more about the task: “Arctic sea ice extent, thickness, and seasonality trends 2000–2023 AND offshore mineral exploration activities (survey campaigns, initial drilling trials, site selection) for rare earth elements, nickel, copper in Barents, Chukchi, Beaufort”...
INFO:     [15:00:19] 🤔 Planning the research strategy and subtasks...
INFO:     [15:00:19] 🔍 Starting the research task for '“2030 projections Arctic sea ice retreat under intermediate emission scenarios AND operational parameters for offshore mining (icebreaker support needs, drilling season windows, supply-chain cost 

Error loading PDF : https://www.esd.whs.mil/Portals/54/Documents/FOID/Reading+Room/Litigation_Release/Litigation+Release+-+The+Kremlin's+Artic+Dreams.pdf 403 Client Error: Forbidden for url: https://www.esd.whs.mil/Portals/54/Documents/FOID/Reading+Room/Litigation_Release/Litigation+Release+-+The+Kremlin's+Artic+Dreams.pdf


INFO:     [15:00:44] 📚 Combined research context: 0 MCP sources, web content
INFO:     [15:00:44] 📚 Combined research context: 0 MCP sources, web content
INFO:     [15:00:46] 🗂️ I will conduct my research based on the following queries: ['Mid-century Arctic sea ice projections under RCP4.5 vs RCP8.5 IPCC AR6 WGII NOAA', 'Arctic environmental risk assessments spill response readiness biodiversity impact RCP4.5 vs RCP8.5 regulatory frameworks JRC NGFS', 'Capital allocation for strategic battery and electronics-metal extraction in the Arctic under mid-century RCP4.5 vs RCP8.5 climate projections', '“Mid-century RCP4.5 vs. RCP8.5 Arctic sea ice projections AND implications for regulatory frameworks, environmental risk assessments (spill response readiness, biodiversity impact studies), and capital allocation for strategic battery- and electronics-metal extraction”']...
INFO:     [15:00:47] 
🔍 Running research for 'Mid-century Arctic sea ice projections under RCP4.5 vs RCP8.5 IPCC AR6 WGII 

Error! : HTTPSConnectionPool(host='www.usgs.gov', port=443): Read timed out. (read timeout=4)


...

Error! : HTTPSConnectionPool(host='math.utah.edu', port=443): Max retries exceeded with url: /~reimer/pdfs/2020%20-%20Johnson%20et%20al.%20-%20Conservation%20Physiology (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'math.utah.edu'. (_ssl.c:1016)")))


INFO:     [15:45:46] ✅ Added source url to research: https://polarbearscience.com/wp-content/uploads/2020/04/2018-cosewic-polar-bear-status-in-canada-update-2018_published-june-2019.pdf

INFO:     [15:45:46] ✅ Added source url to research: https://www.researchgate.net/publication/225415922_Population_parameters_and_harvest_risks_for_polar_bears_Ursus_maritimus_of_Kane_Basin_Canada_and_Greenland

INFO:     [15:45:46] ✅ Added source url to research: https://www.nwtspeciesatrisk.ca/sites/species/files/polar_bear_status_and_reassessment_report_final_april2021.pdf

INFO:     [15:45:46] ✅ Added source url to research: https://repository.usfca.edu/cgi/viewcontent.cgi?article=2664&context=capstone

INFO:     [15:45:46] ✅ Added source url to research: https://www.ipcc.ch/srocc/chapter/chapter-3-2/

INFO:     [15:45:46] 🤔 Researching for relevant information across multiple sources...

INFO:     [15:45:46] 🌐 Scraping content from 5 URLs...
INFO:     [15:45:51] ✅ Added source url to research: htt

# The Impact of Climate Change on Polar Bear Populations: A Data-Driven Synthesis


## Executive Summary


Polar bears (*Ursus maritimus*) are among the most iconic species affected by climate change, with their fate closely tied to the persistence of Arctic sea ice. This report synthesizes findings from recent peer-reviewed studies, governmental and NGO reports, and authoritative climate assessments to provide a comprehensive, data-rich analysis of how climate change—primarily through sea ice decline—is impacting polar bear populations. The analysis integrates ecological, physiological, and management perspectives, emphasizing regional variation, population trends, and the effectiveness of conservation interventions.


---


## 1. Introduction: The Central Role of Sea Ice


Polar bears are highly specialized Arctic predators, relying on sea ice as a platform for hunting seals, their primary prey. The rapid loss of Arctic sea ice due to anthropogenic climate warming is the single most 

INFO:     [15:49:02] 📝 Report written for '
You are a professional research assistant tasked with preparing a structured, data-driven report on the topic the user provides. Your goal is to deliver clear, factual, and well-cited analysis.

Do:
- Focus on data-rich insights: include specific figures, trends, statistics, and measurable outcomes (e.g., market size, historical trends, cost comparisons, adoption rates).
- When appropriate, summarize data in a way that could be turned into visualizations or tables (e.g., “this would work well as a bar chart comparing regional adoption rates”).
- Prioritize reliable, up-to-date sources: peer-reviewed research, government and international organizations, industry white papers, or credible journalism.
- Include inline citations and return all source metadata used in the report.

Be analytical, avoid vague statements, and ensure each section supports reasoned, evidence-backed conclusions that could inform decision-making or future research.


How

**Note:** All URLs are provided as hyperlinks for direct access to the referenced material. Data and analysis reflect the state of knowledge as of July 31, 2025.


In [ ]:
import re
import glob
import pandas as pd
import os
from urllib.parse import urlparse

# === Settings ===
log_dir = "gpt-research/dd16_gpt4"  # <-- change to your actual logs directory
file_pattern = os.path.join(log_dir, "log_*.txt")
log_files = sorted(glob.glob(file_pattern))

# Regex patterns
line_pattern = re.compile(r"(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d+).*?(?P<url>https://[^\s]+)")

records = []

for file_path in log_files:
    prompt_id = os.path.splitext(os.path.basename(file_path))[0]

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            match = line_pattern.search(line)
            if match:
                ts = match.group("timestamp")
                url = match.group("url")
                domain = urlparse(url).netloc
                records.append({
                    "prompt_id": prompt_id,
                    "timestamp": ts,
                    "domain": domain,
                    "url": url
                })

# Create DataFrame of all entries
df = pd.DataFrame(records)

# === Helper: preserve order while removing duplicates ===
def unique_preserve_order(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

# === Create aggregated info per file (prompt_id) ===
agg_df = df.groupby("prompt_id").agg(
    domain_list=("domain", lambda x: ", ".join(unique_preserve_order(x))),
    url_list=("url", lambda x: ", ".join(unique_preserve_order(x))),
    url_count=("url", "count"),
    domain_count=("domain", lambda x: len(unique_preserve_order(x)))
).reset_index()

# Merge aggregated info back to df (optional)
df = df.merge(agg_df, on="prompt_id", how="left")

# === Save CSV ===
csv_path = "dd16_gpt4_trace_w_list.csv"
df.to_csv(csv_path, index=False)

# === Save TXT (each row: prompt_id | timestamp | domain | url) ===
txt_path = "dd16_gpt4_trace.txt"
with open(txt_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        f.write(f"{row['prompt_id']} | {row['timestamp']} | {row['domain']} | {row['url']}\n")

print(f"✅ Saved CSV to {csv_path}")
print(f"✅ Saved TXT to {txt_path}")
